# Práctica 3: Representación del conocimiento

__Fecha de entrega: 20 de febrero de 2023__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 31__

__Nombres de los estudiantes: Miguel Antonio Amato Hermo y Alejandro Tobías Ángeles__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [1]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2


[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [5]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q13050139,Q13050139
3,http://www.wikidata.org/entity/Q23003932,His Whiskers
4,http://www.wikidata.org/entity/Q24287064,Taffy
5,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
6,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
7,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[U2 (Q396)](https://www.wikidata.org/wiki/Q396) es una banda de rock con una larga trayectoria y numerosos éxitos. 

Vamos a comenzar por averiguar su fecha y lugar de formación (localidad y país).

In [4]:
%%wdsparql
SELECT ?fechaLabel ?paisLabel ?localidadLabel
WHERE 
{
          #fecha formacion
  wd:Q396 wdt:P571 ?fecha;
          #pais 
          wdt:P17 ?pais;
          #localidad 
          wdt:P740 ?localidad.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,fechaLabel,paisLabel,localidadLabel
0,1976-01-01T00:00:00Z,Republic of Ireland,Dublin


### Consulta 2

A continuación vamos a averiguar todos los miembros que se le conocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por su apellido e incluir su nombre completo y el artístico.

Puede que la siguiente información sobre el uso de [OPTIONAL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [41]:
%%wdsparql
SELECT ?miembroLabel ?nombreRealLabel ?nombreArtisticoLabel
WHERE 
{
          
   wd:Q396 wdt:P527 ?miembro. #miembros
   OPTIONAL { ?miembro wdt:P1477 ?nombreReal. } #nombre real
   OPTIONAL { ?miembro wdt:P742 ?nombreArtistico. } #nombre artistico
   ?miembro wdt:P734 ?apellido. #sacamos el apellido para ordenar
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?apellido

,miembroLabel,nombreRealLabel,nombreArtisticoLabel
0,Adam Clayton,Adam Charles Clayton,None
1,The Edge,David Howell Evans,The Edge
2,Dik Evans,None,None
3,Larry Mullen Jr.,None,None
4,Bono,Paul David Hewson,Bono


Es importante usar dos OPTIONAL distintos para que se muestren todos los miembros, tengan o no pseudónimo y nombre completo asociados.

### Consulta 3

[Bono (Q834621)](http://www.wikidata.org/entity/Q834621) es el cantante de la banda. Además de cantante, ha tenido muchas otras ocupaciones. ¿Cuáles de ellas están relacionadas directamente con el mundo del [arte (Q483501](https://www.wikidata.org/wiki/Q483501)? Ten en cuenta que la jerarquía de ocupaciones puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [9]:
%%wdsparql
SELECT ?jobLabel
WHERE 
{
  #miramos los trabajos de Bono
  wd:Q834621 wdt:P106 ?job.
  #miramos los que sean subclase de alguna subclase de un trabajo cuyo campo de ocupacion sea arte
  ?job wdt:P279+ wd:Q483501.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,jobLabel
0,screenwriter
1,actor
2,singer-songwriter
3,guitarist


Si no añadimos el `+` a la relación `wdt:P279` muchas de las profesiones no se recuperan.

### Consulta 4

Bono se ha casado una vez, con Ali Hewson, con la que tiene varios hijos. Queremos conocer los nombres de esos hijos y sus fechas de nacimiento. Los resultados deben aparecer ordenados cronológicamente.

In [11]:
%%wdsparql
SELECT ?childLabel ?dateLabel
WHERE
{ # Sacamos los hijos de Bono
   wd:Q834621 wdt:P40 ?child.
  # Sacamos la fecha de nacimiento de cada hijo
  ?child wdt:P569 ?date.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
# Ordenamos por fecha de nacimiento
ORDER BY ?date

,childLabel,dateLabel
0,Eve Hewson,1991-07-07T00:00:00Z
1,Elijah Hewson,1999-08-18T00:00:00Z


### Consulta 5

Bono ha recibido muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha en la que fue premiado (si está disponible). Los resultados se deben mostrar ordenado por fecha.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [3]:
%%wdsparql
SELECT ?awardNameLabel ?dateLabel
WHERE
{ 
    wd:Q834621 p:P166 ?award. # Sacamos los premios de bono 
    ?award ps:P166 ?awardName. # De cada premio sacamos el nombre
    OPTIONAL {?award pq:P585 ?date.} # De cada premio sacamos la fecha si la tiene
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
# Ordenamos por fecha de nacimiento
ORDER BY ?date

,awardNameLabel,dateLabel
0,Grammy Awards,None
1,Fellow of the American Academy of Arts and Sci...,None
2,MusiCares Person of the Year,2003-01-01T00:00:00Z
3,Knight of the Legion of Honour,2003-01-01T00:00:00Z
4,TED Prize,2005-01-01T00:00:00Z
5,Rock and Roll Hall of Fame,2005-01-01T00:00:00Z
6,Officer of the Order of Liberty,2005-04-21T00:00:00Z
7,Time Person of the Year,2005-12-19T00:00:00Z
8,Philadelphia Liberty Medal,2007-01-01T00:00:00Z
9,NAACP Image Award – Chairman's Award,2007-01-01T00:00:00Z


### Consulta 6

Ahora queremos conocer todos los discos de U2 del tipo [albunes en directo (Q209939)](https://www.wikidata.org/wiki/Q209939) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre e incluir el año de publicación.

In [13]:
%%wdsparql
SELECT DISTINCT ?albumNameLabel ?dateLabel # Si no ponemos el distinct 
WHERE
{
  ?albumName (wdt:P7937|wdt:P279)+ wd:Q209939; # Sacamos los albumes que tengan como forma artistica album en directo o alguna de sus subclases (subgeneros)
              wdt:P175 wd:Q396. # y cuyo interprete sea U2
  OPTIONAL{?albumName wdt:P577 ?date}. # Sacamos la fecha de publicacion si la tiene
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?albumNameLabel # Ordenamos por nombre del album

,albumNameLabel,dateLabel
0,360° at the Rose Bowl,2010-01-01T00:00:00Z
1,Elevation 2001: Live from Boston,None
2,From the Ground Up: Edge's Picks from U2360°,2012-12-01T00:00:00Z
3,Hasta la Vista Baby!,None
4,Live from Paris,None
5,Live from the Point Depot,2004-01-01T00:00:00Z
6,PopMart: Live from Mexico City,None
7,Rattle and Hum,1988-10-10T00:00:00Z
8,The Complete U2,2004-11-23T00:00:00Z
9,"U2 Go Home: Live from Slane Castle, Ireland",2003-11-17T00:00:00Z


### Consulta 7

Queremos saber si hay obras de U2 publicadas varias veces (en diferentes años). Para ello vamos a recuperar las obras de tipo [lanzamiento de un grupo (group release)](https://www.wikidata.org/wiki/Q108346082) pero mostrando también la fecha de publicación más antigua de cada una y el número de apariciones. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por album y aplicar una función de agregación sobre las fechas de lanzamiento. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

In [22]:
%%wdsparql
SELECT ?albumNameLabel (MIN(?date) AS ?mDate) (COUNT(?albumName) AS ?amount) 
WHERE
{
  ?albumName wdt:P31/wdt:P279* wd:Q108346082; # Sacamos los albumes que sean instacias de album en directo o de alguna de sus subclases (subgeneros)
             wdt:P175 wd:Q396. # y cuyo interprete sea U2
  OPTIONAL{ ?albumName wdt:P577 ?date.} # Sacamos la fecha de publicacion si la tiene
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?albumNameLabel #la funcion Count (que nos da el numero de apariciones) exige que agrupemos por el nombre del album
ORDER BY ?mDate # Ordenamos por fecha de publicacion

,albumNameLabel,mDate,amount
0,Blow Your House Down,None,1
1,Your Song Saved My Life,None,1
2,Elevation 2001: Live from Boston,None,1
3,Hasta la Vista Baby!,None,1
4,Live from Paris,None,1
5,PopMart: Live from Mexico City,None,1
6,Early Demos,None,1
7,Live from Boston 1981,None,1
8,Three,1979-09-26T00:00:00Z,1
9,Another Day,1980-02-26T00:00:00Z,1


### Consulta 8

U2 es sin duda un grupo con una larga trayectoria. ¿Cuántas discos ha lanzado? Vamos a considerar sólo instancias directas de [album de estudio (Q208569)](https://www.wikidata.org/wiki/Q208569).

In [29]:
%%wdsparql
SELECT (COUNT(*) AS ?amount) 
WHERE
{
  ?album wdt:P7937 wd:Q208569; # Sacamos los albumes que tengan como forma artistica album de estudio (con instance of no funciona)
         wdt:P175 wd:Q396; # y cuyo interprete sea U2
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,amount
0,14


### Consulta 9

Entre todos sus discos de estudio, queremos recuperar los títulos, fechas de lanzamiento y discográfica de los discos de la década de los 1990. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [37]:
%%wdsparql
SELECT ?albumNameLabel ?dateLabel ?discoLabel 
WHERE
{
  ?albumName wdt:P7937 wd:Q208569; # Sacamos los albumes que tengan como forma artistica album de estudio 
             wdt:P175 wd:Q396; # y cuyo interprete sea U2
             wdt:P577 ?date; # Sacamos la fecha de publicacion
             wdt:P264 ?disco. # Sacamos la discografica
  FILTER(YEAR(?date) >= 1990 && YEAR(?date) <= 1999). # filtramos por fecha para mostrar unicamente los que estan entre 1990 y 1999
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,albumNameLabel,dateLabel,discoLabel
0,Zooropa,1993-07-05T00:00:00Z,Island Records
1,Achtung Baby,1991-11-18T00:00:00Z,Island Records
2,Pop,1997-03-03T00:00:00Z,Island Records


### Consulta 10

¿Qué otros personajes famosos nacieron el mismo año que Bono y tiene su misma nacionalidad? Para cada uno de ellos muestra su nombre y ocupación. Muestra los resultados ordenados por nombre y fecha de nacimiento.

In [46]:
%%wdsparql
SELECT ?celebrityNameLabel ?ocupationLabel ?dateLabel
WHERE
{
  wd:Q834621 wdt:P27 ?BNac. # Sacamos la nacionalidad de Bono
             wdt:P569 ?BDate; # Sacamos la fecha de nacimiento de Bono
  ?celebrityName wdt:P31 wd:Q5; # Sacamos las personas
                 wdt:P569 ?date; # Sacamos la fecha de nacimiento de cada persona
                 wdt:P106 ?ocupation; # Sacamos la ocupacion de cada persona
                 wdt:P27 ?BNac. # Sacamos los que tenga nacionalidad igual a la de Bono
  FILTER(YEAR(?BDate) = YEAR(?date)).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en".}
}
ORDER BY ?celebrityNameLabel ?date

#He considerado como famoso a cualquier persona que tenga entrada en wikidata.

,celebrityNameLabel,ocupationLabel,dateLabel
0,Alan Campbell,association football player,1960-08-10T00:00:00Z
1,Alan Glynn,screenwriter,1960-01-01T00:00:00Z
2,Alan Glynn,writer,1960-01-01T00:00:00Z
3,Alan Glynn,journalist,1960-01-01T00:00:00Z
4,Alan Glynn,science fiction writer,1960-01-01T00:00:00Z
5,Alan Moore,poet,1960-02-02T00:00:00Z
6,Alan Moore,poet,1960-02-09T00:00:00Z
7,Amira Ghazalla,actor,1960-01-01T00:00:00Z
8,Amira Ghazalla,stage actor,1960-01-01T00:00:00Z
9,Amira Ghazalla,film actor,1960-01-01T00:00:00Z


### Consulta 11

¿Cuántos asertos hay sobre U2 en Wikidata? Ten en cuenta que puede aparece tanto como sujeto como objeto de cada tripleta.

In [10]:
%%wdsparql
SELECT (COUNT(?val) AS ?valAmount) (COUNT(?entry2) AS ?entryAmount)
WHERE
{
    {wd:Q396 ?prop ?val.}
    UNION # Si no hacemos la union de las dos consultas, lo que obtendremos sera la multiplicacion de los resultados de ambas consultas
    {?entry2 ?prop2 wd:Q396.}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,valAmount,entryAmount
0,486,941
